In [ ]:
from copy import copy

import numpy as np
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.comments import Comment
from openpyxl.styles import Font
from openpyxl.utils.dataframe import dataframe_to_rows
from tqdm import tqdm

df = pd.read_excel("data/table.xlsx", sheet_name="컬럼정의서")
tables = pd.read_excel("data/table.xlsx", sheet_name="valid_list")

In [ ]:
def str_replace(x):
    replace_dict = {
        "CHAT": "CHAR",
        "MUMBER": "NUMBER",
        "VARHCAR2": "VARCHAR2",
        "CAHR": "CHAR",
        "VHARCHAR2": "VARCHAR2",
        "VACHAR2": "VARCHAR2",
        "VARHCHAR2": "VARCHAR2",
        "CHARP": "CHAR",
        "NYNBER": "NUMBER",
        "VAHRCAR2": "VARCHAR2",
        "NUBMER": "NUMBER",
        "VARCAHR2": "VARCHAR2",
    }

    if x is np.nan:
        return x

    for k, v in replace_dict.items():
        x = x.replace(k, v)

    return x


df = df.rename(
    columns={
        "No..1": "SEQ",
        "데이터베이스": "DB",
        "스키마": "SCHEMA",
        "원본 테이블": "테이블/파일/구조체 명",
        "컬럼명": "필드",
        "한글 포함 여부": "한글포함여부",
        "KEY": "Key",
        "상세": "설명",
    }
)

df["DB"] = df["DB"].str.strip()
df["DB"] = df["DB"].str.upper()

df["SCHEMA"] = df["SCHEMA"].str.strip()
df["SCHEMA"] = df["SCHEMA"].str.upper()

df["테이블/파일/구조체 명"] = df["테이블/파일/구조체 명"].str.strip()
df["테이블/파일/구조체 명"] = df["테이블/파일/구조체 명"].str.upper()

df["타입"] = df["데이터 타입 (데이터 길이)"].str.split("(", expand=True)[0]
df["타입"] = df["타입"].apply(str_replace)
df["길이"] = (
    df["데이터 타입 (데이터 길이)"].str.split("(", expand=True)[1].str.replace(")", "")
)
df["NotNull"] = df["제약"].str.contains("NOT NULL").map({True: "O", np.nan: np.nan})
df = df[
    [
        "SEQ",
        "DB",
        "SCHEMA",
        "테이블/파일/구조체 명",
        "필드",
        "한글포함여부",
        "Key",
        "NotNull",
        "타입",
        "길이",
        "설명",
    ]
]
df["Mapping SEQ"] = df["SEQ"]


tables = tables.loc[tables["VALID"].isin(["O", "o"])]
del tables["VALID"]

In [ ]:
class ExcelWriter:
    def __init__(self):
        template_wb = load_workbook("./data/template.xlsx")
        self.header_ws = template_wb["header"]
        self.footer_ws = template_wb["footer"]
        self.data_font = Font(size=10)

        self.wb = Workbook()

    def set_header(self, ws):
        for row in self.header_ws.iter_rows():
            for cell in row:
                new_cell = ws.cell(row=cell.row, column=cell.column, value=cell.value)
                if cell.has_style:
                    new_cell.font = copy(cell.font)
                    new_cell.border = copy(cell.border)
                    new_cell.fill = copy(cell.fill)
                    new_cell.number_format = (
                        cell.number_format
                    )  # 문자열이므로 copy 불필요
                    new_cell.protection = copy(cell.protection)
                    new_cell.alignment = copy(cell.alignment)

                # 메모 복사
                if cell.comment:
                    new_cell.comment = Comment(
                        text=cell.comment.text, author=cell.comment.author
                    )

        # 헤더의 열 너비 복사
        for col_letter, col_dimension in self.header_ws.column_dimensions.items():
            ws.column_dimensions[col_letter].width = col_dimension.width

        # 헤더의 행 높이 복사
        for row_idx, row_dimension in self.header_ws.row_dimensions.items():
            ws.row_dimensions[row_idx].height = row_dimension.height

        # 헤더의 병합된 셀 복사
        for merged_range in self.header_ws.merged_cells.ranges:
            ws.merge_cells(str(merged_range))

    def set_footer(self, ws):
        # 푸터 복사
        footer_start_row = ws.max_row + 1
        for row in self.footer_ws.iter_rows():
            for cell in row:
                new_row = cell.row + footer_start_row - 1
                new_cell = ws.cell(row=new_row, column=cell.column, value=cell.value)
                # 스타일 복사
                if cell.has_style:
                    new_cell.font = copy(cell.font)
                    new_cell.border = copy(cell.border)
                    new_cell.fill = copy(cell.fill)
                    new_cell.number_format = cell.number_format
                    new_cell.protection = copy(cell.protection)
                    new_cell.alignment = copy(cell.alignment)
                # 메모 복사
                if cell.comment:
                    new_cell.comment = Comment(
                        text=cell.comment.text, author=cell.comment.author
                    )

        # 푸터의 열 너비 복사
        for col_letter, col_dimension in self.footer_ws.column_dimensions.items():
            if col_letter not in ws.column_dimensions:
                ws.column_dimensions[col_letter].width = col_dimension.width
            else:
                ws.column_dimensions[col_letter].width = max(
                    ws.column_dimensions[col_letter].width, col_dimension.width
                )

        # 푸터의 행 높이 복사
        for row_idx, row_dimension in self.footer_ws.row_dimensions.items():
            new_row_idx = row_idx + footer_start_row - 1
            ws.row_dimensions[new_row_idx].height = row_dimension.height

        # 푸터의 병합된 셀 복사
        for merged_range in self.footer_ws.merged_cells.ranges:
            min_col, min_row, max_col, max_row = merged_range.bounds
            offset = footer_start_row - 1
            ws.merge_cells(
                start_row=min_row + offset,
                start_column=min_col,
                end_row=max_row + offset,
                end_column=max_col,
            )

    def set_data(self, ws, df):
        # 헤더의 마지막 행 번호
        header_end_row = self.header_ws.max_row

        # DataFrame 내용을 헤더 아래에 추가
        for r_idx, row in enumerate(
            dataframe_to_rows(df, index=False, header=False), start=header_end_row + 1
        ):
            for c_idx, value in enumerate(row, start=1):
                new_cell = ws.cell(row=r_idx, column=c_idx, value=value)
                new_cell.font = self.data_font

    def add(self, sheet_name, df):
        ws = self.wb.create_sheet(title=sheet_name)

        self.set_header(ws)
        self.set_data(ws, df)
        self.set_footer(ws)

    def save(self, output_path):
        try:
            del self.wb["Sheet"]
        except Exception as E:
            print(E)

        self.wb.save(output_path)

In [ ]:
for db_name in tables["DB"].unique():
    schemas = tables.loc[tables["DB"].str.contains(db_name)]["SCHEMA"].unique()

    for schema_name in schemas:
        writer = ExcelWriter()

        table_list = tqdm(
            (
                tables.loc[tables["DB"].str.contains(db_name)]
                .loc[tables["SCHEMA"].str.contains(schema_name)]["TABLE"]
                .unique()
            )
        )

        for table_name in table_list:
            table_list.set_description(f"{db_name}.{schema_name}.{table_name}")

            __selected = (
                df.loc[df["DB"].str.contains(db_name)]
                .loc[df["SCHEMA"].str.contains(schema_name)]
                .loc[df["테이블/파일/구조체 명"] == table_name][
                    [
                        "SEQ",
                        "테이블/파일/구조체 명",
                        "필드",
                        "한글포함여부",
                        "Key",
                        "NotNull",
                        "타입",
                        "길이",
                        "설명",
                        "Mapping SEQ",
                    ]
                ]
            )

            # 테이블이 비어있는 지 체크
            if __selected.empty:
                print(f"{db_name}.{schema_name} {table_name} is empty")
                continue

            writer.add(
                table_name,
                __selected,
            )

        writer.save(f"./data/output/{db_name}_{schema_name}.xlsx")